In [87]:
import pandas as pd
import json
import datetime

In [64]:
file_path = "data/weather_data.json"

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [88]:
data_weather = pd.DataFrame()
start_date = datetime.datetime(2018, 8, 1)


for el in data:
    data_temp = el['main']
    timestamp = start_date
    start_date += timedelta(days=1)
    temp = data_temp['temp']-273
    press = data_temp['pressure']
    humid = data_temp['humidity']
    new_row = {'Timestamp' : timestamp,'temp':temp,'press':press,'humid':humid}

    data_weather = data_weather.append(new_row, ignore_index=True)

C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dat

In [89]:
data_weather

,Timestamp,temp,press,humid
0,2018-08-01,17.70,1008,50
1,2018-08-02,17.70,1008,50
2,2018-08-03,17.72,1008,50
3,2018-08-04,17.68,1008,50
4,2018-08-05,17.68,1008,50
...,...,...,...,...
411,2019-09-16,17.68,1008,50
412,2019-09-17,17.72,1008,50
413,2019-09-18,17.68,1008,50
414,2019-09-19,17.70,1008,50


In [93]:
from datetime import datetime 

In [97]:
building_consumption = pd.read_csv("data/Office building.csv")
building_production = pd.read_csv("data/PV plant.csv")

print(f"building_consumption COLUMNS: {building_consumption.columns}")
print(f"building_production COLUMNS: {building_production.columns}")

data_series = []

for index, row in building_consumption.iterrows():
    timestamp_string = row['Timestamp']
    # Split the string into time and timezone parts
    time_part, timezone_part = timestamp_string.split("+")

    # Split the time part into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_part.split(":"))

    #dt = datetime(row['Year'], row['Month'], row['Day'], hours, minutes, seconds)
    dt = datetime(row['Year'], row['Month'], row['Day'])

    data_series.append(dt)
building_consumption['Timestamp'] = data_series


avg_power_consum = building_consumption.groupby('Timestamp')['Power (W)'].mean()
avg_power_consum = (pd.DataFrame(avg_power_consum))

building_consumption COLUMNS: Index(['Year', 'Month', 'Day', 'Timestamp', 'Power (W)', 'Energy (Wh)'], dtype='object')
building_production COLUMNS: Index(['Year', 'Month', 'Day', 'Timestamp', 'Power (W)'], dtype='object')


In [98]:
data_series = []
for index, row in building_production.iterrows():
    timestamp_string = row['Timestamp']
    # Split the string into time and timezone parts
    time_part, timezone_part = timestamp_string.split("+")

    # Split the time part into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_part.split(":"))

    #dt = datetime(row['Year'], row['Month'], row['Day'], hours, minutes, seconds)
    dt = datetime(row['Year'], row['Month'], row['Day'])

    data_series.append(dt)
building_production['Timestamp'] = data_series

avg_power_product = building_production.groupby('Timestamp')['Power (W)'].mean()
avg_power_product = (pd.DataFrame(avg_power_product))

In [99]:
df_merged_power_consum = avg_power_consum.merge(data_weather, on='Timestamp')
df_merged_power_product = avg_power_product.merge(data_weather, on='Timestamp')

In [102]:
df_merged_power_consum

,Timestamp,Power (W),temp,press,humid
0,2018-08-01,98808.995833,17.70,1008,50
1,2018-08-02,97220.543750,17.70,1008,50
2,2018-08-03,93552.778472,17.72,1008,50
3,2018-08-04,63763.216667,17.68,1008,50
4,2018-08-05,60577.715278,17.68,1008,50
...,...,...,...,...,...
411,2019-09-16,84668.967361,17.68,1008,50
412,2019-09-17,82818.023611,17.72,1008,50
413,2019-09-18,82077.905556,17.68,1008,50
414,2019-09-19,87105.688889,17.70,1008,50
